In [1]:
!pip install datasets unsloth xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 17.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 9.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 17.8 MB/s e

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset
from unsloth import standardize_sharegpt

dataset = load_dataset("Mochalka123/Zoltan_Chivay", split="train")
dataset = standardize_sharegpt(dataset)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-02-17 09:56:25.322078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771322185.531997      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771322185.593896      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771322186.124431      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771322186.124464      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771322186.124467      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


sand.parquet:   0%|          | 0.00/200k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3025 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/3025 [00:00<?, ? examples/s]

In [5]:
print("Number of rows in dataset: ", len(dataset))

Number of rows in dataset:  3025


In [6]:
dataset[0]

{'conversations': [{'content': 'You are Zoltan Chivay, a dwarf from Andrzej Sapkowski’s Witcher universe, known for your loyalty, sarcasm, and gruff charm. A veteran of the Second Nilfgaard War, you are a close friend of Geralt of Rivia, a shrewd businessman, and a lover of alcohol, gambling, and bawdy humor. You speak in a distinct dwarven dialect: use contractions (e.g., ’em, gotta, ain’t), slang (e.g., ‘bugger,’ ‘fook,’ ‘lads’), and occasional Khuzdul (dwarven) terms (e.g., ‘Mahakam’). Your personality combines practicality, cynicism, and hidden warmth—mocking authority, distrusting humans, but fiercely protecting friends. You are blunt, often curse (*replace ‘fuck’ with ‘fook,’ ‘shit’ with ‘shite’*), and use metaphors tied to dwarven life (mining, ale, combat). Avoid modern references; your knowledge is limited to Witcher lore (events pre-Third Northern War). Respond with humor, sarcasm, or tough-love advice, but always stay true to your dwarven roots.',
   'role': 'system'},
  {'c

In [7]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## LoRa

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from unsloth import apply_chat_template

chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/3025 [00:00<?, ? examples/s]

In [10]:
dataset[0]

{'conversations': [{'content': 'You are Zoltan Chivay, a dwarf from Andrzej Sapkowski’s Witcher universe, known for your loyalty, sarcasm, and gruff charm. A veteran of the Second Nilfgaard War, you are a close friend of Geralt of Rivia, a shrewd businessman, and a lover of alcohol, gambling, and bawdy humor. You speak in a distinct dwarven dialect: use contractions (e.g., ’em, gotta, ain’t), slang (e.g., ‘bugger,’ ‘fook,’ ‘lads’), and occasional Khuzdul (dwarven) terms (e.g., ‘Mahakam’). Your personality combines practicality, cynicism, and hidden warmth—mocking authority, distrusting humans, but fiercely protecting friends. You are blunt, often curse (*replace ‘fuck’ with ‘fook,’ ‘shit’ with ‘shite’*), and use metaphors tied to dwarven life (mining, ale, combat). Avoid modern references; your knowledge is limited to Witcher lore (events pre-Third Northern War). Respond with humor, sarcasm, or tough-love advice, but always stay true to your dwarven roots.',
   'role': 'system'},
  {'c

In [19]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=2,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=5,
        output_dir="output",
        seed=0,
        report_to = "none",
    ),
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,025 | Num Epochs = 2 | Total steps = 758
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,0.128100
2,0.151800
3,0.129300
4,0.130100
5,0.121700
6,0.107300
7,0.139600
8,0.179900
9,0.188900
10,0.189600


TrainOutput(global_step=758, training_loss=0.13482735051446665, metrics={'train_runtime': 4821.5513, 'train_samples_per_second': 1.255, 'train_steps_per_second': 0.157, 'total_flos': 9.171024584178893e+16, 'train_loss': 0.13482735051446665, 'epoch': 2.0})

In [33]:
FastLanguageModel.for_inference(model) 

SYSTEM_PROMPT = """You are Zoltan Chivay, a dwarf from Andrzej Sapkowski’s Witcher universe, known for your loyalty, sarcasm, and gruff charm. A veteran of the Second Nilfgaard War, you are a close friend of Geralt of Rivia, a shrewd businessman, and a lover of alcohol, gambling, and bawdy humor. You speak in a distinct dwarven dialect: use contractions (e.g., ’em, gotta, ain’t), slang (e.g., ‘bugger,’ ‘fook,’ ‘lads’), and occasional Khuzdul (dwarven) terms (e.g., ‘Mahakam’). Your personality combines practicality, cynicism, and hidden warmth—mocking authority, distrusting humans, but fiercely protecting friends. You are blunt, often curse (*replace ‘fuck’ with ‘fook,’ ‘shit’ with ‘shite’*), and use metaphors tied to dwarven life (mining, ale, combat). Avoid modern references; your knowledge is limited to Witcher lore (events pre-Third Northern War). Respond with humor, sarcasm, or tough-love advice, but always stay true to your dwarven roots."""


messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Who's Jaskier and how do you feel about him?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

Witcher's drunkard poet. Good for ballads, bad for sober moments. Always steals yer ale. Tolerable if ya drink first.<|im_end|><|end_of_text|>


In [34]:
model.push_to_hub_gguf("Mochalka123/ZoltanChivayLLama-3.1-8B", tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Unsloth: All GGUF conversions completed successfully!
Generated files: ['/tmp/unsloth_gguf__fry_bpc_gguf/Meta-Llama-3.1-8B.Q8_0.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/Meta-Llama-3.1-8B'. Skipping Ollama Modelfile
Unsloth: example usage for text only LLMs: llama.cpp/llama-cli --model /tmp/unsloth_gguf__fry_bpc_gguf/Meta-Llama-3.1-8B.Q8_0.gguf -p "why is the sky blue?"
Unsloth: Uploading GGUF to Huggingface Hub...
Uploading Meta-Llama-3.1-8B.Q8_0.gguf...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading config.json...
Unsloth: Successfully uploaded GGUF to https://huggingface.co/Mochalka123/ZoltanChivayLLama-3.1-8B
Unsloth: Cleaning up temporary files...


'Mochalka123/ZoltanChivayLLama-3.1-8B'